In [21]:
SECONDS_PER_EPOCH = None
SECONDS_PER_EPOCH = 8

OVERLAP = 0.5

In [29]:
import glob, mne
import numpy as np
import pandas as pd

def segment_file(path):
    orig_epochs = mne.read_epochs(path, preload=True)

    if SECONDS_PER_EPOCH:
        data = orig_epochs.get_data()
        data = np.concatenate(data, axis=1)

        raw = mne.io.RawArray(data, orig_epochs.info)
        epochs = mne.make_fixed_length_epochs(raw, duration=SECONDS_PER_EPOCH, overlap=OVERLAP * SECONDS_PER_EPOCH, preload=True)

    epochs.metadata = pd.DataFrame({k: [v[0]] * len(epochs) for k, v in orig_epochs.metadata.items()})

    epochs.save(path.replace("-epo.fif", f"_seg_{SECONDS_PER_EPOCH or 0}s-epo.fif"), overwrite=True)


paths = glob.glob("../data/fif/S??-epo.fif")

for path in paths:
    segment_file(path)

Reading /Users/jakubblaha/Documents/thesis/thesis-python/notebooks/../data/fif/S01-epo.fif ...
    Found the data of interest:
        t =       0.00 ...   14992.19 ms
        0 CTF compensation matrices available
Adding metadata with 3 columns
12 matching events found
No baseline correction applied
0 projection items activated
Creating RawArray with float64 data, n_channels=14, n_times=23040
    Range : 0 ... 23039 =      0.000 ...   179.992 secs
Ready.
Not setting metadata
44 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 44 events and 1024 original time points ...
0 bad epochs dropped
Adding metadata with 3 columns
Overwriting existing file.
Overwriting existing file.
